In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import itertools

In [10]:
def basis_generator(tot_sites, N_e): # Checked OK

  basis = []

  for combination_indices in itertools.combinations(range(tot_sites), N_e):
    state = [0] * tot_sites
    for index in combination_indices:
      state[index] = 1
    basis.append(state)
  
  return np.array(basis, dtype=np.float64)


def hamiltonian_generator_w_int_diff_V(basis_set, tot_sites, v, w, V1, V2):
  d = len(basis_set)
  H = np.zeros((d, d))

  for i in range(d):
    for j in range(d):
      H[j][i] = np.dot(hopping_on_ket_w_int_diff_V(basis_set[i], tot_sites, v, w), basis_set[j])
  
  for idx, state in enumerate(basis_set):
        energy = 0.0
        for i in range(len(state) - 1):
          if i % 2 == 0:
            energy += V1 * state[i] * state[i+1]
          elif i % 2 != 1:
            energy += V2 * state[i] * state[i+1]
        H[idx, idx] += energy

  return H


def hopping_on_ket_w_int_diff_V(ket, tot_sites, v, w):
  res = np.zeros((tot_sites), dtype=np.float64)
  for i in range(tot_sites):
    if i % 2 == 0 and i - 1 > 0:
      if ket[i - 1] == 1:
        res[i] += w
      if ket[i + 1] == 1:
        res[i] += v
    elif i % 2 == 1 and i + 1 < tot_sites:
      if ket[i - 1] == 1:
        res[i] += v
      if ket[i + 1] == 1:
        res[i] += w
    elif i == 0 and ket[i + 1] == 1:
      res[i] += v
    elif i == tot_sites - 1 and ket[i - 1] == 1:
      res[i] += v
  return res # normalize(res)

In [12]:
N = 2
w = 2
v = 1
V1 = 1
V2 = 2
tot_sites = 2 * N
N_e = N

basis_set = basis_generator(tot_sites, N_e)

for i in range(len(basis_set)):
    ket = basis_set[i]
    print(f'{ket} ------ {hopping_on_ket_w_int_diff_V(ket, tot_sites, v, w)}')

hamiltonian = hamiltonian_generator_w_int_diff_V(basis_set, tot_sites, v, w, V1, V2)

print(hamiltonian)
print(sp.linalg.ishermitian(hamiltonian))


[1. 1. 0. 0.] ------ [1. 1. 2. 0.]
[1. 0. 1. 0.] ------ [0. 3. 0. 1.]
[1. 0. 0. 1.] ------ [0. 1. 1. 0.]
[0. 1. 1. 0.] ------ [1. 2. 2. 1.]
[0. 1. 0. 1.] ------ [1. 0. 3. 0.]
[0. 0. 1. 1.] ------ [0. 2. 1. 1.]
[[3. 3. 1. 3. 1. 2.]
 [3. 0. 1. 3. 4. 1.]
 [1. 1. 0. 2. 1. 1.]
 [3. 3. 2. 4. 3. 3.]
 [1. 4. 1. 3. 0. 3.]
 [2. 1. 1. 3. 3. 3.]]
True
